### Key Features Implemented:

1. **Auto Star Selection**:
   - The script uses `camera.GuiderAutoSelectStar = True` to automatically select the brightest guide star in the field.
   - If you want manual control over which star is selected, you can disable auto-selection by setting `GuiderAutoSelectStar` to `False` and manually specify coordinates using `GuiderSetStarPosition`.

2. **Aggressiveness Settings**:
   - The aggressiveness settings for both X and Y axes are set to **8.0** using `GuiderAggressivenessX` and `GuiderAggressivenessY`.

3. **Exposure Time**:
   - The script takes an exposure of **15 seconds** using `camera.GuiderExpose(15)`.

4. **Tracking Data Logging**:
   - After each exposure, it retrieves the following data:
     - Star position (`GuiderXStarPosition`, `GuiderYStarPosition`)
     - Guiding errors (`GuiderXError`, `GuiderYError`)
     - Corrections applied based on aggressiveness settings (`corr_x`, `corr_y`)
     - Brightness of the guide star (`MaxPixel`)
   - This information is logged into a file (`tracking_log.txt`) with timestamps in Universal Time (UT).

5. **Real-Time Monitoring**:
   - The script prints tracking information to both the console and a log file for real-time monitoring.

6. **Graceful Exit**:
   - The script runs continuously until interrupted by pressing `Ctrl+C`. When interrupted, it stops autoguiding using `camera.GuiderStop()`.



### Python Code for Autoguiding with MaxIm DL:


In [ ]:
import win32com.client
import time
from datetime import datetime

# Initialize MaxIm DL connection
camera = win32com.client.Dispatch("MaxIm.CCDCamera")

# Enable camera link
camera.LinkEnabled = True

# Check if camera is connected and ready
if camera.CameraStatus == "Idle":
    print("Camera connected successfully.")
else:
    print(f"Camera status: {camera.CameraStatus}")
    exit()

# Set autoguider to automatically select the brightest star
camera.GuiderAutoSelectStar = True

# If manual star selection is needed, disable auto-selection and set initial star position (in pixels)
# camera.GuiderAutoSelectStar = False
# initial_x = 5910.761  # Example X coordinate of guide star
# initial_y = 3062      # Example Y coordinate of guide star
# camera.GuiderSetStarPosition(int(initial_x), int(initial_y))

# Set X and Y aggressiveness to 8.0 as requested
camera.GuiderAggressivenessX = 8.0
camera.GuiderAggressivenessY = 8.0

print("Autoguider setup complete with auto star selection and aggressiveness set.")

# Function to start guiding with a 15-second exposure and log tracking data
def start_autoguiding():
    log_file = 'tracking_log.txt'
    
    # Open file in append mode to log tracking data
    with open(log_file, 'a') as file:
        # Write header if it's a new log
        file.write(f"Track Log Started at UT {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')}\n")
        file.write("UT Time,   Star X,  Star Y, OffsetX, OffsetY,  Corr X,  Corr Y,  Bright\n")
        
        try:
            while True:
                # Start autoguiding exposure for 15 seconds
                camera.GuiderExpose(15)
                print("Autoguiding started with a 15-second exposure.")

                # Wait for exposure to complete (add a small buffer time)
                time.sleep(16)

                # Ensure that new measurements are available before reading errors
                if not camera.GuiderNewMeasurement:
                    print("No new guider measurements available.")
                    continue

                # Get current star position and errors
                star_x = camera.GuiderXStarPosition   # X position of guide star (pixels)
                star_y = camera.GuiderYStarPosition   # Y position of guide star (pixels)
                x_error = camera.GuiderXError         # Error in X axis (pixels)
                y_error = camera.GuiderYError         # Error in Y axis (pixels)
                corr_x = camera.GuiderAggressivenessX * x_error  # Correction applied in X axis (seconds)
                corr_y = camera.GuiderAggressivenessY * y_error  # Correction applied in Y axis (seconds)

                # Get brightness of the guide star (you may need to adjust this based on your setup)
                brightness = camera.MaxPixel

                # Get current date and time in UT
                timestamp = datetime.utcnow().strftime('%H:%M:%S')

                # Log tracking data to file
                file.write(f"{timestamp}, {star_x:.2f}, {star_y:.2f}, {x_error:.2f}, {y_error:.2f}, {corr_x:.2f}, {corr_y:.2f}, {brightness}\n")
                
                # Print the logged data to console for real-time monitoring
                print(f"{timestamp} - Star X: {star_x:.2f}, Star Y: {star_y:.2f}, OffsetX: {x_error:.2f}, OffsetY: {y_error:.2f}, Corr X: {corr_x:.2f}, Corr Y: {corr_y:.2f}, Brightness: {brightness}")

                # Wait for 15 seconds before starting the next exposure (adjust as needed)
                time.sleep(15)

        except KeyboardInterrupt:
            # Stop autoguiding when the user interrupts the script (Ctrl+C)
            print("Autoguiding stopped.")
            camera.GuiderStop()

# Start the autoguiding process
start_autoguiding()